In [1]:
listpath = '..\TestRun2\TrackingList.csv'
# This is currently the most up to date tracking list version

In [3]:
import requests
import pandas as pd
from datetime import datetime as dt
from xml.etree import ElementTree as ET
from config import userid, password
from bs4 import BeautifulSoup

base_url = 'https://secure.shippingapis.com/ShippingAPI.dll?API=TrackV2&XML='

tracking_df = pd.read_csv(listpath,dtype={'Client Nos.':str})

# Build a list of delivered packages based on API calls and save it to a file:
charlotte_list = []
filename = 'Output\charlottelist.txt'
charlottefile = open(filename, 'a')
charlottefile.write(
    'Undelivered items that went through Charlotte:\n'
)
for num in tracking_df['Tracking Nos.']:
    response = requests.get(base_url + \
                        f'<TrackRequest USERID="{userid}" PASSWORD="{password}">\
                            <TrackID ID="{num}"></TrackID>\
                                </TrackRequest>')
    soup = BeautifulSoup(response.content, 'xml')

    if 'charlotte' in str(response.content).lower():
        charlotte_list.append(num)
        spot = tracking_df[tracking_df['Tracking Nos.'] == num].index[0]
        # NOTE: the above line is written with the assumption that exactly one row is returned. If the condition returns more than one index for a tracking number, the first will go into the varible.
        charlottefile.write(f'\n{tracking_df.iloc[spot,0]}, {tracking_df.iloc[spot,1]}, {tracking_df.iloc[spot,2]}')
        charlottefile.write('\n\n' + soup.prettify() + '\n')

charlottefile.close()

print(f'{len(charlotte_list)} undelivered files went through Charlotte.')

76 undelivered files went through Charlotte.
